Published on November 20, 2025. By Marília Prata, mpwolke

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt


#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_2b_en/3/config.json
/kaggle/input/gemma/keras/gemma_2b_en/3/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/3/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/3/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/3/assets/tokenizer/vocabulary.spm
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/reference.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/AIMO3_Reference_Problems.pdf
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/aimo_3_inference_server.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/aimo_3_gateway.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/__init__.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation/core/templates.py
/kaggle/input/ai-mathematical-olympiad-pr

![](https://png.pngtree.com/png-clipart/20211116/original/pngtree-doodle-math-element-for-decoration-png-image_6930898.png)PngTree

## Competition Citation:

@misc{ai-mathematical-olympiad-progress-prize-3,

    author = {Simon Frieder and Sam Bealing and Philip Vonderlind and Sida Li and Arsenii Nikolaiev and Geoff C. Smith and Kevin Buzzard and Timothy Gowers and Peter J. Liu and Po-Shen Loh and Lester Mackey and Leonardo de Moura and Dan Roberts and D. Sculley and Terence Tao and David Balduzzi and Simon Coyle and Alex Gerko and Ryan Holbrook and Addison Howard and XTX Markets},
    
    title = {AI Mathematical Olympiad - Progress Prize 3},
    year = {2025},
    howpublished = {\url{https://kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3}},
    note = {Kaggle}
}

## Submission file

In [2]:
sub = pd.read_csv('../input/ai-mathematical-olympiad-progress-prize-3/sample_submission.csv', delimiter=',', encoding='cp1251')
sub.tail()

,id,answer
0,000aaa,0
1,111bbb,0
2,222ccc,0


## All the Reference csv file

In [3]:
ref = pd.read_csv('../input/ai-mathematical-olympiad-progress-prize-3/reference.csv')
ref.tail(10)

,id,problem,answer
0,0e644e,Let $ABC$ be an acute-angled triangle with int...,336
1,26de63,Define a function $f \colon \mathbb{Z}_{\geq 1...,32951
2,424e18,A tournament is held with $2^{20}$ runners eac...,21818
3,42d360,"On a blackboard, Ken starts off by writing a p...",32193
4,641659,"Let $ABC$ be a triangle with $AB \neq AC$, cir...",57447
5,86e8e5,Let $n \geq 6$ be a positive integer. We call ...,8687
6,92ba6a,Alice and Bob are each holding some integer nu...,50
7,9c1c5f,Let $f \colon \mathbb{Z}_{\geq 1} \to \mathbb{...,580
8,a295e9,A $500 \times 500$ square is divided into $k$ ...,520
9,dd7f5e,Let $\mathcal{F}$ be the set of functions $\al...,160


## The third AIMO Progress Prize competition

AI Mathematical Olympiad - Progress Prize 3

"The goal of this competition is to create open-source algorithms and models that can solve olympiad-level math problems written in LaTeX format. Your participation will help to advance AI models’ mathematical reasoning skills and drive frontier knowledge."

https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3/overview

"The competition data comprises 110 mathematics problems. The answer to each problem is a non-negative integer between 0 and 99999, inclusive.
All problems are text-only with mathematical notation in LaTeX. Although some problems may involve geometry, diagrams are not used in any problem."

https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3/data

### The dollar sign ($) that insists Not to show AGAIN on the test.csv

According to Taichi Uemura answer on Prize 1. "A possible reason is as follows:

"When a text is truncated at middle of a math formula (i.e. between two dollar signs), MathJax doesn't recognize it as a math formula, so the dollar sign is left as is. Some train texts are indeed truncated with pandas default options. Test texts are short, so they are not truncated."

https://www.kaggle.com/competitions/ai-mathematical-olympiad-prize/discussion/488297

In [4]:
test = pd.read_csv('../input/ai-mathematical-olympiad-progress-prize-3/test.csv', delimiter=',', encoding='cp1251')
test.tail()

,id,problem
0,000aaa,What is $1-1$?
1,111bbb,What is $0\times10$?
2,222ccc,Solve $4+x=4$ for $x$.


## Install Keras

In [5]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatib

2025-11-20 22:16:36.508424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763676996.883708      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763676996.983251      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Gemma Keras 2b_en

In [6]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

2025-11-20 22:18:46.266663: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [7]:
#By York Yong https://www.kaggle.com/code/yorkyong/gemma-trial-ace-a-data-science-interview

aimo_dataset = []
    
for index, row in ref.iterrows():
    question, answer = row['problem'], row['answer']
    template = (f"problem:\n{question}\n\nanswer:\n{answer}")
    aimo_dataset.append(template)

## LoRA

In [8]:
# Enable LoRA for the model and set the LoRA rank to 64.
gemma_lm.backbone.enable_lora(rank=64)

## Control Memory Usage

In [9]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

## Epochs - GPU H100

Upgraded Kaggle Hardware

We're excited to announce that we've added powerful **new H100 machines** to the AIMO hardware pool! See this page for more on these machines.

What you need to know:

**AIMO 3 Only** - H100 are only available for notebooks attached to this competition, use of H100s for any other activity could result in moderation action (e.g. site suspension or account ban).

**No Internet** - To help ensure these machines are reserved for competition use, all H100 sessions must have internet disabled.

In [10]:
%%time

gemma_lm.fit(aimo_dataset, epochs=1, batch_size=1)

10/10 ━━━━━━━━━━━━━━━━━━━━ 756s 73s/step - loss: 0.6073 - sparse_categorical_accuracy: 0.6193
CPU times: user 44min 39s, sys: 1min 48s, total: 46min 27s
Wall time: 12min 38s


### Find the remainder 

In [11]:
ref['problem'][0]

'Let $ABC$ be an acute-angled triangle with integer side lengths and $AB<AC$. Points $D$ and $E$ lie on segments $BC$ and $AC$, respectively, such that $AD=AE=AB$. Line $DE$ intersects $AB$ at $X$. Circles $BXD$ and $CED$ intersect for the second time at $Y \\neq D$. Suppose that $Y$ lies on line $AD$. There is a unique such triangle with minimal perimeter. This triangle has side lengths $a=BC$, $b=CA$, and $c=AB$. Find the remainder when $abc$ is divided by $10^{5}$.'

### What is the remainder...

In [12]:
ref['problem'][5]

'Let $n \\geq 6$ be a positive integer. We call a positive integer $n$-Norwegian if it has three distinct positive divisors whose sum is equal to $n$. Let $f(n)$ denote the smallest $n$-Norwegian positive integer. Let $M=3^{2025!}$ and for a non-negative integer $c$ define \n\\begin{equation*}\n    g(c)=\\frac{1}{2025!}\\left\\lfloor \\frac{2025! f(M+c)}{M}\\right\\rfloor.\n\\end{equation*}\nWe can write \n\\begin{equation*}\n    g(0)+g(4M)+g(1848374)+g(10162574)+g(265710644)+g(44636594)=\\frac{p}{q}\n\\end{equation*}\nwhere $p$ and $q$ are coprime positive integers. What is the remainder when $p+q$ is divided by $99991$?'

### No two of these rectangles have the same perimeter, the largest possible value is...

In [13]:
ref['problem'][8]

'A $500 \\times 500$ square is divided into $k$ rectangles, each having integer side lengths. Given that no two of these rectangles have the same perimeter, the largest possible value of $k$ is $\\mathcal{K}$. What is the remainder when $k$ is divided by $10^{5}$?'

### Two functions $\\alpha$ and $\\beta$

In [14]:
ref['problem'][9]

'Let $\\mathcal{F}$ be the set of functions $\\alpha \\colon \\mathbb{Z}\\to \\mathbb{Z}$ for which there are only finitely many $n \\in \\mathbb{Z}$ such that $\\alpha(n) \\neq 0$. \n\nFor two functions $\\alpha$ and $\\beta$ in $\\mathcal{F}$, define their product $\\alpha\\star\\beta$ to be $\\sum\\limits_{n\\in\\mathbb{Z}} \\alpha(n)\\cdot \\beta(n)$. Also, for $n\\in\\mathbb{Z}$, define a shift operator $S_n \\colon \\mathcal{F}\\to \\mathcal{F}$ by $S_n(\\alpha)(t)=\\alpha(t+n)$ for all $t \\in \\mathbb{Z}$.\n\nA function $\\alpha \\in \\mathcal{F}$ is called \\emph{shifty} if \n\\begin{itemize}\n    \\item $\\alpha(m)=0$ for all integers $m<0$ and $m>8$ and\n    \\item There exists $\\beta \\in \\mathcal{F}$ and integers $k \\neq l$ such that for all $n \\in \\mathbb{Z}$\n    \\begin{equation*}\n        S_n(\\alpha)\\star\\beta =\n        \\begin{cases}\n            1 & n \\in \\{k,l\\} \\\\\n            0 & n \\not \\in \\{k,l\\}\n        \\end{cases}\n        \\; .\n    \\end{

### Alice and Bob holding some integer number of sweets

In [15]:
ref['problem'][6]

"Alice and Bob are each holding some integer number of sweets. Alice says to Bob: ``If we each added the number of sweets we're holding to our (positive integer) age, my answer would be double yours. If we took the product, then my answer would be four times yours.'' Bob replies: ``Why don't you give me five of your sweets because then both our sum and product would be equal.'' What is the product of Alice and Bob's ages?"

### Another remainder question

In [16]:
ref['problem'][3]

'On a blackboard, Ken starts off by writing a positive integer $n$ and then applies the following move until he first reaches $1$. Given that the number on the board is $m$, he chooses a base $b$, where $2 \\leq b \\leq m$, and considers the unique base-$b$ representation of $m$,\n\\begin{equation*}\n    m = \\sum_{k = 0}^\\infty a_k \\cdot b^k\n\\end{equation*}\nwhere $a_k$ are non-negative integers and $0 \\leq a_k < b$ for each $k$. Ken then erases $m$ on the blackboard and replaces it with $\\sum\\limits_{k = 0}^\\infty a_k$.\n\nAcross all choices of $1 \\leq n \\leq 10^{10^5}$, the largest possible number of moves Ken could make is $M$. What is the remainder when $M$ is divided by $10^{5}$?'

## What is 1-1$?   

I asked that 2y ago. Let's check how Gemma performs it now.

In [17]:
%%time
print(gemma_lm.generate("What is $1-1$?", max_length=256))

What is $1-1$?

Step 1
1 of 2

$1-1=0 $

Result
2 of 2

$0 $
CPU times: user 2min 48s, sys: 3.08 s, total: 2min 51s
Wall time: 1min 17s


### What is the largest possible number of moves Ken could make?

In [18]:
%%time
print(gemma_lm.generate("What's the largest possible number of moves that Ken could make?", max_length=256))

What's the largest possible number of moves that Ken could make?

What's the smallest possible number of moves that Ken could make?

What's the largest possible number of moves that Ken could make?

What's the smallest possible number of moves that Ken could make?

What's the largest possible number of moves that Ken could make?

What's the smallest possible number of moves that Ken could make?

What's the largest possible number of moves that Ken could make?

What's the smallest possible number of moves that Ken could make?

What's the largest possible number of moves that Ken could make?

What's the smallest possible number of moves that Ken could make?

What's the largest possible number of moves that Ken could make?

What's the smallest possible number of moves that Ken could make?

What's the largest possible number of moves that Ken could make?

What's the smallest possible number of moves that Ken could make?

What's the largest possible number of moves that Ken could make?

Wha

### What are the side lengths of a unique triangle with minimal perimeter?

In [19]:
%%time
print(gemma_lm.generate("What are the side lengths of a unique triangle with minimal perimeter?", max_length=256))

What are the side lengths of a unique triangle with minimal perimeter?

Step 1
1 of 2

The perimeter of a triangle is the sum of the lengths of its sides.

The triangle with the minimal perimeter is a right triangle.

The length of the hypotenuse is the square root of the sum of the squares of the other two sides.

$\begin{align*} \text{Hypotenuse}&=\sqrt{a^2+b^2}\\ &=\sqrt{1^2+2^2}\\ &=\sqrt{1+4}\\ &=\sqrt{5}\\ \end{align*} $

Result
2 of 2

The length of the hypotenuse is $\sqrt{5}$.
CPU times: user 6min 41s, sys: 2.87 s, total: 6min 44s
Wall time: 3min 57s


#### A square with $500 \\times 500$ 

In [20]:
%%time
print(gemma_lm.generate("What's the perimeter of a square $500 \\times 500$?", max_length=256))

What's the perimeter of a square $500 \times 500$?

Step 1
1 of 2

The perimeter of a square is the sum of the lengths of all its sides.

The length of a side of a square is equal to the length of the diagonal of the square.

The diagonal of a square is the hypotenuse of a right triangle with legs of length $500$ and $500$.

The length of the diagonal of a square is given by the formula:

$\begin{align*} d&=\sqrt{a^2+b^2} \end{align*} $

where $a$ and $b$ are the lengths of the legs of the right triangle.

Substitute $a=500$ and $b=500$ in the formula:

$\begin{align*} d&=\sqrt{500^2+500^2}\\ &=\sqrt{250000+250000}\\ &=\sqrt{500000}\\ &=500\sqrt{2} \end{align*} $

The perimeter of the square is then:

$\begin{
CPU times: user 10min 37s, sys: 3.96 s, total: 10min 41s
Wall time: 6min 25s


### Define the product of two functions $\\alpha$ and $\\beta$ (ref 9)

In [21]:
%%time
print(gemma_lm.generate("Define the product of two functions $\\alpha$ and $\\beta$", max_length=256))

Define the product of two functions $\alpha$ and $\beta$ to be $\alpha \beta(x)=\int_{-\infty}^{\infty} \alpha(x-y) \beta(y) d y$. Show that if $\alpha$ and $\beta$ are both even functions, then $\alpha \beta$ is an even function.

Step 1
1 of 2

Let $\alpha$ and $\beta$ be even functions. Then,

$\begin{align*} \alpha(x-y) &= \alpha(x-y) \quad \text{for all } x,y \in \mathbb{R} \\ \beta(y) &= \beta(y) \quad \text{for all } y \in \mathbb{R} \end{align*} $

Therefore,

$\begin{align*} \alpha \beta(x) &= \int_{-\infty}^{\infty} \alpha(x-y) \beta(y) \, dy \\ &= \int_{-\infty}^{\infty} \alpha(x-y) \beta(y) \, dy \\ &= \int_{-\infty}^{\infty} \alpha(x-y) \beta(y) \, dy \\ &= \int_{-\
CPU times: user 10min 40s, sys: 3.29 s, total: 10min 44s
Wall time: 6min 26s


## What's the remainder of this Kaggle Notebook Gemma?

#### Rhetorical question.

![](https://i0.wp.com/streetartutopia.com/wp-content/uploads/2021/10/Math-with-Bart-Simpson-Street-Art-by-One-Mizer-1.jpg?fit=1080%2C1080&ssl=1)Strret Art Utopia

#Acknowledgements:

AI Google Developers https://ai.google.dev/gemma/docs/lora_tuning#load_dataset

York Yong https://www.kaggle.com/code/yorkyong/gemma-trial-ace-a-data-science-interview

Marília Prata https://www.kaggle.com/code/mpwolke/hut3-hut5-hut8-hut13-fibonacci-got-it-gemma/notebook

Marília Prata https://www.kaggle.com/code/mpwolke/what-is-1-1-gemma/notebook